In [18]:
] activate ../../Acceleration

 Activating environment at `~/Desktop/Code/Acceleration/Acceleration/Project.toml`


In [35]:
using Acceleration
using LinearAlgebra

n = 3000; # number of samples
d = 30;    # number of features
selected_labels = [0,1,2,3];  ## Only two labels allowed

data, labels = load_dataset("MNIST",[0,1],n,d);
f,∇f,∇ϕ_cjg,σ,L,μ  = linear_regression(data,labels);
μ2 = μ;



┌ Info: Loading dataset MNIST with 60000 samples and 784 features
└ @ Acceleration /Users/victor/Desktop/Code/Acceleration/Acceleration/src/datasets.jl:26


In [36]:
n = size(data,2)
x_ini = zeros(n)

using JuMP
using Ipopt

model = Model(with_optimizer(Ipopt.Optimizer))
set_silent(model)
@variable(model, x[1:n])
@objective(model, Min, f(x))
@constraint(model, con, 0 .<= x)
@time optimize!(model)
x_opt = value.(x);

  0.022354 seconds (1.69 k allocations: 129.844 KiB)


In [37]:
for j in [0.01,0.1,0.5,0.9]
n = length(x_ini)
x = x_ini;
y = zeros(n);
z = zeros(n);
v = zeros(n);
A0 = σ/(2*L);
A = A0;
AX = zeros(n);

k = 2000;
yks = zeros(n,k);
var_ξ = zeros(k)
#μ = 0;
s = zeros(n);
p = 1
ξ = 0;
w = 0;


for i=1:k   
    λ = j;
    a = L-μ; b = -λ*(μ*A + σ); c = -λ*(μ*A^2 + σ*A);
    α = (-b + sqrt(b^2 - 4*a*c)) / (2*a);
    A = A + α;
    x = (A-α)/A*y + α/A*v; 
    AX = AX + α*x;

    q = (rand(n).-0.5)
    η = zeros(n) #1.2*ξ*q/norm(q);
    #println(norm(η))
    approx_gradient = ∇f(x)+η;
    
    s = s + α*η;
    z = z - α*approx_gradient;
    
    v_past = v;
    v = max.(0,(z + μ*AX)*inv(1 + μ*(A-A0)));
    y = (A-α)*inv(A)*y + α*inv(A)*v; 
    ξ = sqrt(1-λ)*(μ*(A-α) + σ)/α * norm(v-v_past,2);
    var_ξ[i] = ξ;
    println(ξ)
      
    yks[:,i] = y;
        
        
end

x_out = yks;
p_out = sqrt.(sum((x_out .- x_opt).^2, dims=1)');


filename = string("logs/",j,".txt")
io = open(filename, "w")
for i=1:k
    print(io,i)
    print(io,"\t")
    print(io, p_out[i])
    print(io,"\t")
    println(io, var_ξ[i])
end
close(io)
end

2267.32688942858
2244.8611539381172
2214.581538802654
2177.8745933354794
2135.840440011403
2089.378687199955
2039.2455341884486
1986.0924780602934
1930.4929795892874
1872.9610699421205
1813.964449224317
1753.9337441207374
1693.2690329945278
1632.3443855073153
1571.510925396203
1511.0987647512222
1451.41804878698
1392.7592750582628
1335.3930002328252
1279.5690149770846
1225.5150499984593
1173.4350722771271
1123.5072393247003
1075.881600299492
1030.6776645433472
987.9819974431196
947.8460448908829
910.284422786655
875.2973239908807
843.2740939504718
813.181773989442
785.3333477644849
759.5630635147526
735.6803500082209
713.4758197012237
692.7278380536183
673.209271576631
654.6940316930031
636.9630814741125
619.8096545517
603.0435327747354
586.4943249903027
570.0137712753886
553.4771581986292
536.7839696566969
519.8579167638707
502.6464935455795
485.1201977781426
467.2715427756295
449.1139698072194
430.68075449562
412.023985154414
393.21367654271273
374.3370676715833
355.4981342993453
336

0.7996396190374512
0.7930232447216718
0.7907622505101134
0.7927255753865322
0.7986887597774588
0.808347202098305
0.821334372269395
0.8372421378033852
0.855640542505108
0.8760950929867765
0.8981805116137893
0.9214907192398827
0.9456453685259352
0.9702935394882469
0.9951152898154944
1.0198217022335123
1.0441539587824722
1.0678818441830829
1.0908019635620056
1.1127358643907914
1.1335281806368984
1.153044865859457
1.171171547509796
1.1878120125482732
1.2028868211576056
1.216332038081506
1.228098067715303
1.238148578229383
1.24645950065319
1.2530180902355736
1.2578220393057005
1.2608786328184034
1.2622039397139169
1.2618220351345668
1.2597642501137605
1.256068446979241
1.2507783199316194
1.2439427214248682
1.2356150159080301
1.2258524633125796
1.2147156352132733
1.2022678671543867
1.188574750895888
1.1737036706084416
1.1577233870570078
1.1407036738441676
1.1227150096179268
1.1038283298919644
1.084114841827857
1.0636459049046545
1.0424929798621485
1.0207276478065475
0.998421700596287
0.97564

0.013839633937769474
0.013366049065720614
0.012952712484975476
0.012602646161066936
0.012317887798780743
0.01209928410470254
0.011946346917565213
0.011857201713119611
0.011828644101354709
0.011856300133260436
0.011934867112476544
0.012058399208411955
0.012220599445687753
0.012415085974549548
0.012635611861766236
0.012876229739210914
0.013131402415696741
0.013396066644223732
0.013665660023882666
0.013936121140892084
0.014203871913796802
0.014465789244212051
0.014719171255910551
0.014961701780057303
0.015191415421717
0.015406664647171503
0.015606089603146382
0.01578859097475923
0.015953305885550734
0.01609958667267406
0.016226982282590632
0.016335222012862013
0.01642420125755966
0.016493969007000824
0.016544716784258253
0.016576768777810832
0.016590572951943633
0.01658669289713093
0.01656580024638131
0.016528667487051733
0.0164761609877104
0.016409234124123546
0.01632892033670401
0.0162363260128136
0.01613262306363978
0.01601904107775149
0.01589685895926604
0.015767395940938008
0.0156320

0.00034460515986895504
0.00034491207516211927
0.00034544106471378196
0.00034619425433308527
0.00034717320320099086
0.00034837864820289077
0.00034981023564935147
0.00035146631414836795
0.0003533437682542827
0.00035543786224863496
0.0003577421799668648
0.00036024855297503205
0.0003629470600889889
0.00036582606950427867
0.0003688722949269299
0.0003720708998371124
0.0003754056207684758
0.00037885890574130827
0.00038241208471714454
0.00038604552944683615
0.0003897388311739652
0.0003934709830306848
0.00039722054211838804
0.0004009658154403131
0.00040468500754120253
0.0004083563802060778
0.00041195839764761533
0.00041546985269036457
0.00041886998990819544
0.0004221386149177702
0.0004252561935375199
0.0004282039326840889
0.0004309638625671489
0.0004335189039065021
0.0004358529176383834
0.0004379507657404862
0.00043979834032973627
0.0004413826087913167
0.0004426916334752736
0.0004437145972256507
0.00044444182112243866
0.00044486477208759037
0.0004449760726544172
0.00044476950477285727
0.0004442

0.0002550861957032086
0.00028234476679202394
0.0003080184765842597
0.00033136917470575445
0.0003519454246199529
0.0003694782066658582
0.00038382136181647057
0.0003949162710353246
0.0004027698297874229
0.0004074399105882775
0.0004090251803538914
0.0004076575449779273
0.00040349625810010947
0.00039672314248496776
0.0003875386108865252
0.00037615831855597335
0.00036281035922831726
0.00034773298264631654
0.00033117286037637124
0.00031338394875448543
0.000294627079825182
0.00027517042826981125
0.0002552911040406894
0.00023527817435608925
0.00021543748551770115
0.00019609860171383245
0.00017762379170377935
0.00016041777639992703
0.00014493397747260828
0.00013166724922252703
0.00012111564879397808
0.00011369348584211552
0.00010960302254985302
0.00010872516645962676
0.00011061385021429475
0.00011461215713621754
0.00012001308901126765
0.00012617850035896784
0.00013259037249592736
0.00013885433498333013
0.000144682148201635
0.00014986971649199007
0.0001542775487568832
0.00015781538573183605
0.00

...

4.5048256533147425e-13
3.863147206893916e-13
1.8775621611275493e-13
3.8667759262428923e-13
2.2763645251463906e-13
1.5299677168916672e-13
1.8921372465434972e-13
1.3419359631684058e-13
3.9603770305148774e-13
3.5556060232258785e-13
3.166321815773403e-13
3.354632812645782e-13
2.820574632570552e-13
3.3732300909612254e-13
3.817839049000233e-13
4.561300853324435e-13
8.377746523562799e-14
1.6668814433379318e-13
4.372921588392816e-13
1.6520248096131726e-13
2.4194375851705217e-13
3.8239023246815364e-13
3.767878992001513e-13
6.068654734239835e-13
3.9970936739407605e-13
3.385373340464208e-13
2.635892455325756e-13
6.012894185226016e-13
6.509263170458205e-13
2.996331368144212e-13
3.312473354169913e-13
4.5147599542192227e-13
1.3247948791259754e-13
3.206037686760138e-13
3.31598410766103e-13
2.1778698370964026e-13
4.65572754682595e-13
2.654390965590245e-13
1.0387642833298102e-13
4.339809516186004e-13
4.100152441435492e-13
3.8207359392934634e-13
4.276558304771758e-13
2.7981653228673446e-13
2.66667879932

2.994803257351886e-13
3.3393460089701957e-13
9.12588085087225e-14
1.5168916313359587e-13
3.67304952068008e-13
2.9510261812153264e-13
5.47522493219233e-13
2.774133689142254e-13
2.1376284725913108e-13
1.5542202136234006e-13
3.0351890674553074e-13
2.992497694422485e-13
3.7577464537473174e-13
3.330343013226331e-13
4.733510076623209e-13
4.3080814544319585e-13
4.79356246074462e-13
2.4030654290955046e-13
3.321018448392774e-13
4.2334896446499573e-13
9.021692872743473e-14
3.4914189750016457e-13
4.3566772077084696e-13
2.670455029245918e-13
3.551849520026689e-13
2.9169923039567255e-13
4.4911192664401313e-13
3.93383329773877e-13
5.175848345047189e-13
4.573060848013183e-13
4.2100755311193807e-13
2.266302008528132e-13
3.863765674002078e-13
2.909224630934309e-13
3.6024985581330566e-13
2.9844675003697503e-13
3.673334115556177e-13
1.883456062822376e-13
4.461365687508037e-13
4.221218966038084e-13
4.387938491400002e-13
1.9995862006628394e-13
1.990186287052268e-13
2.9593160752463484e-13
3.1069214796408177

2.9968684832951065e-8
2.6195663270753043e-8
2.2722122407696008e-8
1.95369730354379e-8
1.6630562872005822e-8
1.3994965748536751e-8
1.162479148589731e-8
9.518630736189265e-9
7.681765833060088e-9
6.130833237760887e-9
4.90006094370536e-9
4.04278323341985e-9
3.604956776447806e-9
3.5603390387907806e-9
3.788962882939656e-9
4.151318005850281e-9
4.549146651057583e-9
4.928083438497372e-9
5.26170787977766e-9
5.538710098339881e-9
5.75589162641486e-9
5.91427856065952e-9
6.0171940550233546e-9
6.069111637167357e-9
6.075043617358707e-9
6.04018262109622e-9
5.969583137338442e-9
5.8681985157871874e-9
5.7406647647620875e-9
5.5913371978445156e-9
5.4241939640914826e-9
5.242889072916048e-9
5.050714717741264e-9
4.850616476231748e-9
4.645280226119904e-9
4.436954960616002e-9
4.227695743535422e-9
4.019299124891945e-9
3.813181580379307e-9
3.610678669332201e-9
3.412800650457131e-9
3.2203956694586584e-9
3.034180157419083e-9
2.854635088796983e-9
2.68216295067243e-9
2.5170003705345217e-9
2.3592885691354044e-9
2.20913

1.7739152645686876e-13
1.8499337947843486e-13
1.1291753448267909e-13
1.685246890885639e-13
3.5442882112409786e-14
4.3397783155025214e-14
1.3852208427492855e-13
1.6841500349986738e-13
3.8750202494739507e-14
5.236397453786664e-14
1.5935217522985944e-13
1.8509538718644286e-13
5.011955877187959e-14
1.6221599677253343e-13
1.7625864403357982e-13
1.3184837538822237e-13
1.1798857122137693e-13
1.7875218283025118e-13
1.808340516647086e-13
1.7972997485009428e-13
1.0685358663490799e-13
1.3677462128601923e-13
1.9688291062639616e-13
1.2789389013682982e-13
1.6296992699012135e-13
9.717662722591794e-14
1.708871745953683e-13
2.28709945425688e-13
3.926375681971213e-14
4.414085969278233e-14
1.1093887392884434e-13
3.0256406223849844e-13
4.827405316429801e-13
2.8807573506057216e-13
1.5645955828192936e-13
4.993195991956964e-14
1.479071592444293e-13
4.132198451068922e-13
2.1957262115995468e-13
1.283240142204328e-13
2.4268298624471827e-13
2.7972995862549817e-13
3.2186314756604145e-13
2.699945807044833e-13
2.49

1.7218512521974758e-13
3.9591021173571453e-14
1.4267215205724987e-13
1.7498405947045785e-13
2.5752449230496395e-13
1.269715750248938e-13
4.160884427536346e-14
2.8674731381076464e-13
1.100176860461549e-13
1.137641361189783e-13
2.0920652549146298e-13
1.431494542364652e-13
1.615471810587857e-13
1.1670826831144238e-13
1.5969301790997603e-13
3.0552892534915983e-13
5.734113039692624e-14
1.419285517884564e-13
2.0188835514041028e-13
1.7135874367845713e-13
1.1232136293738248e-13
1.1072773123234239e-13
1.970331675907158e-13
1.7392299549893342e-13
1.8311370996797262e-13
4.500533180087574e-14
1.4216665560544311e-13
1.4687999254419738e-13
8.394113792549978e-14
1.5310091712615807e-13
2.858346369529854e-13
1.4063356570347105e-13
3.334539946998875e-13
1.4448056567631837e-13
6.089114027423395e-14
5.29609583826548e-14
1.1352600949568921e-13
1.640521959720295e-13
9.837304790563869e-14
4.643566390526737e-14
4.025994829621852e-14
5.020737364634262e-14
1.4079553613492158e-13
1.8964533245289432e-13
2.0544622

In [40]:
using Plots
using LinearAlgebra

p_out = sqrt.(sum((x_out .- x_opt).^2, dims=1)');

plot(max.(hcat(p_out,var_ξ),1e-5),yaxis=:log,linewidth=2,xlabel="training rounds",ylabel="loss")
#plot(max.(hcat(p_out),1e-20),yaxis=:log)

LoadError: UndefVarError: x_out not defined

In [39]:
∇f(x_opt)

30-element Array{Float64,1}:
  2.4601013137726113e-7
  3.878962108672113e-5
  3.4832118132044343e-6
 68.29703762402463
 81.9083684352324
  9.273862193737159e-6
  3.528501508753834e-7
  5.14369446591445e-7
 10.244003591298883
  8.864235994110459e-7
  4.627554233138653e-7
 37.397131732258444
  9.14806358309761e-6
  ⋮
  6.008766766818496e-6
 14.826901798646894
 16.781369377831144
  1.0244983315033096e-6
  4.459286991243932
 21.568909020290313
  5.704242833620291
 25.15106247782506
 13.317349277532621
  4.381550865071858
  7.007751321452815e-7
  2.390725135710177